In [2]:
# Config
# Tables
import pandas as pd
# Charting
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Math
import numpy as np
# Tools
import os

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [3]:
empresas = {
    'DVO':      { 'factor': 1.6,  'cargo': 0.45 },
    'MILANI':   { 'factor': 1.6,  'cargo': 0.45 },
    'MIDJ':     { 'factor': 1.6,  'cargo': 0.45 },
    'ARPER':    { 'factor': 1.6,  'cargo': 0.45 },
    'OMP':      { 'factor': 3.2,  'cargo': 0.45 },
    'MOHAWK':   { 'factor': 2.85, 'cargo': 0.43 },
    'VICCARBE': { 'factor': 1.3,  'cargo': 0.61 },
    'SUNON':    { 'factor': 4,    'cargo': 0.66 },
    'MARTE':    { 'factor': 1.6,  'cargo': 0.60 },
    'KASTEL':   { 'factor': 1.6, 'cargo': 0.45 },
    'CONFISA':  { 'factor': 1.6, 'cargo': 0.66 },
    'EUN':      { 'factor': 1.6, 'cargo': 0.30 },}
emp = pd.DataFrame.from_dict(empresas).T
emp


,factor,cargo
DVO,1.60,0.45
MILANI,1.60,0.45
MIDJ,1.60,0.45
ARPER,1.60,0.45
OMP,3.20,0.45
MOHAWK,2.85,0.43
VICCARBE,1.30,0.61
SUNON,4.00,0.66


In [4]:
def load_data():
       df = pd.read_excel('./data/Ventas.xlsx')
       df = df[['MARCA', 'Nº Producto Sistema', 'Nº referencia cruzada', 'Descripción',
              'Cantidad', 'PL / FOB UNITARIO', 'Grupo_Producto']]
       df = pd.merge(df,emp, left_on='MARCA',right_index=True)

       df.rename(columns={'PL / FOB UNITARIO':'FOB'},inplace=True)

       df['PV'] = df['FOB'] * (df['factor'] + df['cargo'])
       df['Ganancias'] = df['PV'] - df['FOB']
       df['Cargo Maritimo'] = df['FOB'] * df['cargo']
       df['Costo'] = df['FOB'] + df['Cargo Maritimo']


       return df

df = load_data()
df

,MARCA,Nº Producto Sistema,Nº referencia cruzada,Descripción,Cantidad,FOB,Grupo_Producto,factor,cargo,PV,Ganancias,Cargo Maritimo,Costo
2,DVO,DVO-00747,REU. 9P 280X100,Mesa Reunion 2.80x1.00,1,1449.000000,Mesa,1.6,0.45,2970.450000,1521.450000,652.05,2101.050000
3,DVO,DVO-00771,ESCREJEC802PS160X0.6,Escritorio 160x0.65 Mueble Auxiliar,1,1490.000000,Escritorio,1.6,0.45,3054.500000,1564.500000,670.50,2160.500000
4,DVO,DVO-00458,BENCH 6P 1.60,ESTACIONES TIPO BENCH 6P 1.60*0.65,1,2915.000000,Bancada,1.6,0.45,5975.750000,3060.750000,1311.75,4226.750000
5,DVO,DVO-00322,550CS2004#WN,CAJONERA 2 GAV. CON FILE BOX,6,238.000000,Caja,1.6,0.45,487.900000,249.900000,107.10,345.100000
6,DVO,DVO-00023,NaN,Credenza Gavetas 2 Class. L900 H827,4,694.000000,Credenza,1.6,0.45,1422.700000,728.700000,312.30,1006.300000
7,DVO,DVO-00177,CREDENZA 0.90 #WN,Credenza 0.90 X 0.45,3,349.888889,Credenza,1.6,0.45,717.272222,367.383333,157.45,507.338889
8,DVO,DVO-00585,NaN,Mesa de Reunion 200X74X90,1,983.000000,Mesa,1.6,0.45,2015.150000,1032.150000,442.35,1425.350000
9,DVO,DVO-00585,NaN,Mesa de Reunion 200X74X90,1,1070.000000,Mesa,1.6,0.45,2193.500000,1123.500000,481.50,1551.500000
10,DVO,DVO-00774,902TM3620,Mesa con Alargo Abierto 200 Vigo,1,3126.000000,Mesa,1.6,0.45,6408.300000,3282.300000,1406.70,4532.700000


In [5]:
df.sum()

C:\Users\Dasa\AppData\Local\Temp\ipykernel_552\1703867807.py:1: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



MARCA                                        DVODVODVODVODVODVODVODVODVO
Nº Producto Sistema    DVO-00747DVO-00771DVO-00458DVO-00322DVO-00023D...
Descripción            Mesa Reunion 2.80x1.00Escritorio 160x0.65 Mueb...
Cantidad                                                              19
FOB                                                         12314.888889
Grupo_Producto         MesaEscritorioBancadaCajaCredenzaCredenzaMesaM...
factor                                                              14.4
cargo                                                               4.05
PV                                                          25245.522222
Ganancias                                                   12930.633333
Cargo Maritimo                                                    5541.7
Costo                                                       17856.588889
dtype: object

In [69]:
def treemap(df):
    df = df.copy()    

    df = pd.melt(df, id_vars=['MARCA','Grupo_Producto'], value_vars=['FOB','Cargo Maritimo','Ganancias'],
        var_name='Tipo', value_name='Valor')
    df['Valor'] = df['Valor'].astype(int)
    df = df.groupby(by=['MARCA','Grupo_Producto','Tipo']).sum().reset_index()

    df['Proceso'] = np.where(df['Tipo']=='Ganancias','Ganancias','Costo')
    fig = px.treemap(df, path=['Proceso','Tipo','MARCA','Grupo_Producto'], values='Valor', color='MARCA', hover_name='Valor',)
    fig.update_traces(root_color="lightgrey")
    fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
    fig.show()

treemap(df)

In [152]:
def discount_graph(df, descuento=10):
    def percent(num,percent):
        num = num * (100 - percent) / 100
        return num
    
    df = df.copy()
    sums = df[['PV','Costo Total','Ganancias']].sum()
    start = sums[0]
    end = sums[1]

    descuentos = []
    roi = []
    ventas = []
    ganancias = []
    gastos = []
    for i in range(100):
        y = percent(start, i)
        g = y-end
        roi.append((g*100/end))
        descuentos.append(i)
        ventas.append(y)
        ganancias.append(g)
        gastos.append(end)
        if y < end:
            break

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(
        go.Scatter(x=descuentos, y=gastos, name="Gastos", fill=None, marker_color='blue'),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=descuentos, y=ventas, name="Ganancias", fill='tonexty', marker_color='crimson'),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=[descuento], y=[gastos[descuento]], mode='markers+text', 
            text=[f'{int(gastos[descuento]):,}'],
            textposition='bottom right',
            marker_color='blue',
            showlegend=False
            ),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=descuentos, y=roi, name="Retorno de Inversion", marker_color='gold'),
        secondary_y=True,
    )
    fig.add_trace(
        go.Scatter(x=[descuento,descuento],y=[0,start], name='Descuento', mode='lines', line=dict(dash='dash'),marker_color='black',),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=[descuento], y=[ventas[descuento]], mode='markers+text', 
            text=[f'{int(ventas[descuento]):,}'],
            textposition='top right',
            marker_color='crimson',
            showlegend=False
            ),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=[descuento], y=[roi[descuento]], mode='markers+text', 
            text=[f'{round(roi[descuento])}%'],
            textposition='top right',
            marker_color='gold',
            showlegend=False
            ),
        secondary_y=True,
    )

    # Add figure title
    fig.update_layout(
        title_text=f"Precio y Retorno por Descuento\nGanancias: {int(ganancias[descuento]):,}"
    )
    # fig.add_hline(y=end)


    # Set x-axis title
    fig.update_xaxes(title_text="Descuento", ticks='inside', showgrid=False, ticksuffix='%')

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Ventas</b> USD$", ticks='inside',secondary_y=False, showgrid=False, rangemode='tozero')
    fig.update_yaxes(title_text="<b>Retorno</b> de Inveriosn", ticks='inside', ticksuffix='%',secondary_y=True, showgrid=False, title_font=dict(color='gold'))

    fig.show()
discount_graph(df, descuento=12)